### Instalaciones

In [ ]:
!pip install mediapipe
!pip install opencv-python
!pip install matplotlib

### Imports

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import math
from matplotlib import pyplot as plt


In [ ]:
NOSE = 0
LEFT_EYE_INNER = 1
LEFT_EYE = 2
LEFT_EYE_OUTER = 3
RIGHT_EYE_INNER = 4
RIGHT_EYE = 5
RIGHT_EYE_OUTER = 6
LEFT_EAR = 7
RIGHT_EAR = 8
MOUTH_LEFT = 9
MOUTH_RIGHT = 10
LEFT_SHOULDER = 11
RIGHT_SHOULDER = 12
LEFT_ELBOW = 13
RIGHT_ELBOW = 14
LEFT_WRIST = 15
RIGHT_WRIST = 16
LEFT_PINKY = 17
RIGHT_PINKY = 18
LEFT_INDEX = 19
RIGHT_INDEX = 20
LEFT_THUMB = 21
RIGHT_THUMB = 22
LEFT_HIP = 23
RIGHT_HIP = 24
LEFT_KNEE = 25
RIGHT_KNEE = 26
LEFT_ANKLE = 27
RIGHT_ANKLE = 28
LEFT_HEEL = 29
RIGHT_HEEL = 30
LEFT_FOOT_INDEX = 31
RIGHT_FOOT_INDEX = 32
  
total_joints = {
    0 : "NOSE", 1 : "LEFT_EYE_INNER",2 : "LEFT_EYE",3 : "LEFT_EYE_OUTER",
    4 : "RIGHT_EYE_INNER",5 : "RIGHT_EYE",6 : "RIGHT_EYE_OUTER",
    7 : "LEFT_EAR",8 : "RIGHT_EAR",9 : "MOUTH_LEFT",
    10 : "MOUTH_RIGHT",11 : "LEFT_SHOULDER",12 : "RIGHT_SHOULDER",
    13 : "LEFT_ELBOW",14 : "RIGHT_ELBOW",15 : "LEFT_WRIST",
    16 : "RIGHT_WRIST",17 : "LEFT_PINKY",18 : "RIGHT_PINKY",
    19 : "LEFT_INDEX",20 : "RIGHT_INDEX",21 : "LEFT_THUMB",
    22 : "RIGHT_THUMB",23 : "LEFT_HIP",24 : "RIGHT_HIP",
    25 : "LEFT_KNEE",26 : "RIGHT_KNEE",27 : "LEFT_ANKLE",
    28 : "RIGHT_ANKLE",29 : "LEFT_HEEL",30 : "RIGHT_HEEL",
    31 : "LEFT_FOOT_INDEX",32 : "RIGHT_FOOT_INDEX"
}

key_joints = {
    11 : "LEFT_SHOULDER",
    12 : "RIGHT_SHOULDER",
    13 : "LEFT_ELBOW",
    14 : "RIGHT_ELBOW",
    15 : "LEFT_WRIST",
    16 : "RIGHT_WRIST",
    23 : "LEFT_HIP",
    24 : "RIGHT_HIP",
    25 : "LEFT_KNEE",
    26 : "RIGHT_KNEE",
    27 : "LEFT_ANKLE",
    28 : "RIGHT_ANKLE",
    29 : "LEFT_HEEL",
    30 : "RIGHT_HEEL",
}

key_joints_idx = [11,12,13,14,15,16,23,24,25,26,27,28,29,30]

visibility_threshold = 0.9
circle_radius = 30
circle_color = (128, 0, 255)
circle_left = (255,0,0)
circle_right = (0,255,0)
image_array = []


In [ ]:


def distance_2(p1:(float,float, float), p2:(float,float, float)):
    distance = math.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2 + ((p2[2] - p1[2])**2))
    return distance

class Joint:
    def __init__(self, x_, y_, z_, visibility_, id_):
        self.x = x_
        self.y = y_
        self.z = z_
        self.visibility = visibility_
        self.id = id_
    
class Point:
    
    def __init__(self, x_, y_, z_):
        self.x = x_
        self.y = y_
        self.z = z_

class Pose:
    
    def __init__(self, l_shoulder_ : (float,float, float), r_shoulder_ : (float,float, float)):
        self.l_shoulder = l_shoulder_
        self.r_shoulder = r_shoulder_
        self.key_length = distance_2(self.l_shoulder,self.r_shoulder)

    def __init__(self, key_point_:Point):
        self.key_point = key_point_
        self.joints = {
        "LEFT_SHOULDER":None,
        "RIGHT_SHOULDER":None,
        "LEFT_ELBOW":None,
        "RIGHT_ELBOW":None,
        "LEFT_WRIST":None,
        "RIGHT_WRIST":None,
        "LEFT_HIP":None,
        "RIGHT_HIP":None,
        "LEFT_KNEE":None,
        "RIGHT_KNEE":None,
        "LEFT_ANKLE":None,
        "RIGHT_ANKLE":None,
        "LEFT_HEEL":None,
        "RIGHT_HEEL":None,
        }
    
    def add_joint(self, joint_ : Joint):
        joint_name = key_joints[joint_.id]
        self.joints[joint_name] = joint_
        
    def add_circles_independent(self, image, joints):
    
        for joint_name,joint in joints.items():
            if joint.visibility > visibility_threshold:
                if joint_name == "LEFT_SHOULDER":
                    cv2.circle(image, (int(joint.x),int(joint.y)), circle_radius,  circle_left  , -1) 
                elif joint_name == "RIGHT_SHOULDER":
                    cv2.circle(image, (int(joint.x),int(joint.y)), circle_radius,  circle_right  , -1)
                else:
                 cv2.circle(image, (int(joint.x),int(joint.y)), circle_radius,  circle_color  , -1)
                            
    def project_pose(self, angle):
        
        alt_joints = dict(self.joints)
        rad_angle =  math.radians(angle)
        blank1 = cv2.imread("blank.jpg")
        blank_rgb1 = cv2.cvtColor(blank1,cv2.COLOR_BGR2RGB)
        
        for name, joint in alt_joints.items():
            new_x= self.key_point.x + (joint.x - self.key_point.x) * math.cos(rad_angle) - (joint.z - self.key_point.z) * math.sin(rad_angle)
            new_z = self.key_point.z + (joint.x- self.key_point.x) * math.sin(rad_angle) + (joint.z - self.key_point.z) * math.cos(rad_angle)
            new_joint = Joint(new_x, joint.y, new_z, joint.visibility, joint.id)
            alt_joints[name] = new_joint
            
        self.add_circles_independent(blank_rgb1, alt_joints)
        
        cv2.line(blank_rgb1, (int(alt_joints["LEFT_SHOULDER"].x) , int(alt_joints["LEFT_SHOULDER"].y)), (int(alt_joints["RIGHT_SHOULDER"].x ), int(alt_joints["RIGHT_SHOULDER"].y)), circle_color, 10)
        
        cv2.line(blank_rgb1, (int(alt_joints["LEFT_SHOULDER"].x) , int(alt_joints["LEFT_SHOULDER"].y)), (int(alt_joints["LEFT_ELBOW"].x ), int(alt_joints["LEFT_ELBOW"].y)), circle_color, 10)
        cv2.line(blank_rgb1, (int(alt_joints["RIGHT_SHOULDER"].x) , int(alt_joints["RIGHT_SHOULDER"].y)), (int(alt_joints["RIGHT_ELBOW"].x ), int(alt_joints["RIGHT_ELBOW"].y)), circle_color, 10)

        cv2.line(blank_rgb1, (int(alt_joints["RIGHT_ELBOW"].x) , int(alt_joints["RIGHT_ELBOW"].y)), (int(alt_joints["RIGHT_WRIST"].x ), int(alt_joints["RIGHT_WRIST"].y)), circle_color, 10)
        cv2.line(blank_rgb1, (int(alt_joints["LEFT_ELBOW"].x) , int(alt_joints["LEFT_ELBOW"].y)), (int(alt_joints["LEFT_WRIST"].x ), int(alt_joints["LEFT_WRIST"].y)), circle_color, 10)

        
        cv2.line(blank_rgb1, (int(alt_joints["LEFT_SHOULDER"].x) , int(alt_joints["LEFT_SHOULDER"].y)), (int(alt_joints["RIGHT_HIP"].x ), int(alt_joints["RIGHT_HIP"].y)), circle_color, 10)
        cv2.line(blank_rgb1, (int(alt_joints["RIGHT_SHOULDER"].x) , int(alt_joints["RIGHT_SHOULDER"].y)), (int(alt_joints["LEFT_HIP"].x ), int(alt_joints["LEFT_HIP"].y)), circle_color, 10)


        img_array.append(blank_rgb1)
        #plt.imshow(blank_rgb1)    
        
        
   
        
        

    def add_circles(self, image):
        
        for joint_name,joint in self.joints.items():
            if joint.visibility > visibility_threshold:
                if joint_name == "LEFT_SHOULDER":
                    cv2.circle(image, (int(joint.x),int(joint.y)), circle_radius,  circle_left  , -1) 
                elif joint_name == "RIGHT_SHOULDER":
                    cv2.circle(image, (int(joint.x),int(joint.y)), circle_radius,  circle_right  , -1)
                else:
                 cv2.circle(image, (int(joint.x),int(joint.y)), circle_radius,  circle_color  , -1)
                   
        

In [ ]:
def obtain_joints(image, image_name):
    mp_drawing = mp.solutions.drawing_utils
    mp_pose = mp.solutions.pose

    with mp_pose.Pose(static_image_mode = True) as pose:
        height, width, _ = image.shape
        image_rgb = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        #blank = cv2.imread("blank.jpg")
        #blank_rgb = cv2.cvtColor(blank,cv2.COLOR_BGR2RGB)
        result = pose.process(image_rgb)
        if result.pose_landmarks is not None:                
            for i in key_joints_idx:
                x1 = int(result.pose_landmarks.landmark[i].x*width)
                y1 = int(result.pose_landmarks.landmark[i].y*height)
                z1 = 0
                visibility = result.pose_landmarks.landmark[i].visibility
                if visibility > visibility_threshold:
                    cv2.circle(image_rgb, (x1,y1), circle_radius, circle_color , -1)   

            cv2.imwrite("marked_" + image_name , image_rgb)
            
            return result


In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
image_name = 'image_2.jpeg'
img_array = []


with mp_pose.Pose(static_image_mode = True) as pose:

    image = cv2.imread(image_name)
    height, width, _ = image.shape
    image_rgb = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)

    blank = cv2.imread("blank.jpg")
    blank_rgb = cv2.cvtColor(blank,cv2.COLOR_BGR2RGB)

    result = pose.process(image_rgb)
    point = Point(int(width/2),int(height/2),0)
    pose = Pose(point)
    
    if result.pose_landmarks is not None:                
        
        for i in key_joints_idx:
                        
            x1 = int(result.pose_landmarks.landmark[i].x*width)
            y1 = int(result.pose_landmarks.landmark[i].y*height)
            z1 = 0
            visibility = result.pose_landmarks.landmark[i].visibility
            print(total_joints[i],visibility)
            joint = Joint(x1,y1,z1,visibility, i)
            pose.add_joint(joint)
            #cv2.circle(image_rgb, (x1,y1), circle_radius, circle_color , -1)
           
        pose.add_circles(image_rgb)
        plt.imshow(image_rgb)
        

    
    for i in range(0,180):
        pose.project_pose(i)
        
    codec = cv2.VideoWriter_fourcc(*'mp4v')     
    out = cv2.VideoWriter('video1.mp4',codec, 36, (width,height))

    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()
    



In [ ]:
cam1 = cv2.VideoCapture(1)
cam2 = cv2.VideoCapture(2)

result1, image1 = cam1.read()
result2, image2 = cam2.read()

name_1 = "image1.png"
name_2 = "image2.png"

img1 = None
img2 = None

if result1 and result2:
    
    # show the image
    cv2.imwrite(name_1, image1)
    cv2.imwrite(name_2, image2)
    img1 = obtain_joints(image1, name_1)
    img2 = obtain_joints(image2, name_2)

cam1.release() 
cam2.release() 







In [ ]:
block_size = 9

stereo = cv2.StereoBM_create(numDisparities=32, blockSize = block_size)

imgLeft = cv2.imread("image1.png",0)
imgRight = cv2.imread("image2.png",0)
disparity = stereo.compute(imgLeft,imgRight)

min = disparity.min()
max = disparity.max()
disparity = np.uint8(255 * (disparity-min) / (max-min))


plt.imshow(disparity)

In [ ]:

# Check for left and right camera IDs
# These values can change depending on the system
CamL_id = 2 # Camera ID for left camera
CamR_id = 0 # Camera ID for right camera

CamL= cv2.VideoCapture(CamL_id)
CamR= cv2.VideoCapture(CamR_id)

# Reading the mapping values for stereo image rectification
cv_file = cv2.FileStorage("stereo_rectify_maps.xml", cv2.FILE_STORAGE_READ)
Left_Stereo_Map_x = cv_file.getNode("Left_Stereo_Map_x").mat()
Left_Stereo_Map_y = cv_file.getNode("Left_Stereo_Map_y").mat()
Right_Stereo_Map_x = cv_file.getNode("Right_Stereo_Map_x").mat()
Right_Stereo_Map_y = cv_file.getNode("Right_Stereo_Map_y").mat()
cv_file.release()

def nothing(x):
    pass

cv2.namedWindow('disp',cv2.WINDOW_NORMAL)
cv2.resizeWindow('disp',600,600)

cv2.createTrackbar('numDisparities','disp',1,17,nothing)
cv2.createTrackbar('blockSize','disp',5,50,nothing)
cv2.createTrackbar('preFilterType','disp',1,1,nothing)
cv2.createTrackbar('preFilterSize','disp',2,25,nothing)
cv2.createTrackbar('preFilterCap','disp',5,62,nothing)
cv2.createTrackbar('textureThreshold','disp',10,100,nothing)
cv2.createTrackbar('uniquenessRatio','disp',15,100,nothing)
cv2.createTrackbar('speckleRange','disp',0,100,nothing)
cv2.createTrackbar('speckleWindowSize','disp',3,25,nothing)
cv2.createTrackbar('disp12MaxDiff','disp',5,25,nothing)
cv2.createTrackbar('minDisparity','disp',5,25,nothing)

# Creating an object of StereoBM algorithm
stereo = cv2.StereoBM_create()

while True:

	# Capturing and storing left and right camera images
	retL, imgL= CamL.read()
	retR, imgR= CamR.read()
	
	# Proceed only if the frames have been captured
	if retL and retR:
		imgR_gray = cv2.cvtColor(imgR,cv2.COLOR_BGR2GRAY)
		imgL_gray = cv2.cvtColor(imgL,cv2.COLOR_BGR2GRAY)

		# Applying stereo image rectification on the left image
		Left_nice= cv2.remap(imgL_gray,
							Left_Stereo_Map_x,
							Left_Stereo_Map_y,
							cv2.INTER_LANCZOS4,
							cv2.BORDER_CONSTANT,
							0)
		
		# Applying stereo image rectification on the right image
		Right_nice= cv2.remap(imgR_gray,
							Right_Stereo_Map_x,
							Right_Stereo_Map_y,
							cv2.INTER_LANCZOS4,
							cv2.BORDER_CONSTANT,
							0)

		# Updating the parameters based on the trackbar positions
		numDisparities = cv2.getTrackbarPos('numDisparities','disp')*16
		blockSize = cv2.getTrackbarPos('blockSize','disp')*2 + 5
		preFilterType = cv2.getTrackbarPos('preFilterType','disp')
		preFilterSize = cv2.getTrackbarPos('preFilterSize','disp')*2 + 5
		preFilterCap = cv2.getTrackbarPos('preFilterCap','disp')
		textureThreshold = cv2.getTrackbarPos('textureThreshold','disp')
		uniquenessRatio = cv2.getTrackbarPos('uniquenessRatio','disp')
		speckleRange = cv2.getTrackbarPos('speckleRange','disp')
		speckleWindowSize = cv2.getTrackbarPos('speckleWindowSize','disp')*2
		disp12MaxDiff = cv2.getTrackbarPos('disp12MaxDiff','disp')
		minDisparity = cv2.getTrackbarPos('minDisparity','disp')
		
		# Setting the updated parameters before computing disparity map
		stereo.setNumDisparities(numDisparities)
		stereo.setBlockSize(blockSize)
		stereo.setPreFilterType(preFilterType)
		stereo.setPreFilterSize(preFilterSize)
		stereo.setPreFilterCap(preFilterCap)
		stereo.setTextureThreshold(textureThreshold)
		stereo.setUniquenessRatio(uniquenessRatio)
		stereo.setSpeckleRange(speckleRange)
		stereo.setSpeckleWindowSize(speckleWindowSize)
		stereo.setDisp12MaxDiff(disp12MaxDiff)
		stereo.setMinDisparity(minDisparity)

		# Calculating disparity using the StereoBM algorithm
		disparity = stereo.compute(Left_nice,Right_nice)
		# NOTE: compute returns a 16bit signed single channel image,
		# CV_16S containing a disparity map scaled by 16. Hence it 
		# is essential to convert it to CV_32F and scale it down 16 times.

		# Converting to float32 
		disparity = disparity.astype(np.float32)

		# Scaling down the disparity values and normalizing them 
		disparity = (disparity/16.0 - minDisparity)/numDisparities

		# Displaying the disparity map
		cv2.imshow("disp",disparity)

		# Close window using esc key
		if cv2.waitKey(1) == 27:
			break
	
	else:
		CamL= cv2.VideoCapture(CamL_id)
		CamR= cv2.VideoCapture(CamR_id)

print("Saving depth estimation paraeters ......")

cv_file = cv2.FileStorage("depth_estmation_params_py.xml", cv2.FILE_STORAGE_WRITE)
cv_file.write("numDisparities",numDisparities)
cv_file.write("blockSize",blockSize)
cv_file.write("preFilterType",preFilterType)
cv_file.write("preFilterSize",preFilterSize)
cv_file.write("preFilterCap",preFilterCap)
cv_file.write("textureThreshold",textureThreshold)
cv_file.write("uniquenessRatio",uniquenessRatio)
cv_file.write("speckleRange",speckleRange)
cv_file.write("speckleWindowSize",speckleWindowSize)
cv_file.write("disp12MaxDiff",disp12MaxDiff)
cv_file.write("minDisparity",minDisparity)
cv_file.write("M",39.075)
cv_file.release()

In [ ]:
# ------------------------------
# Notice
# ------------------------------

# Copyright 1966 Clayton Darwin claytondarwin@gmail.com

# ------------------------------
# Imports
# ------------------------------

import time
import traceback

import numpy as np
import cv2

import targeting_tools as tt

# ------------------------------
# Testing
# ------------------------------

def run():

    # ------------------------------
    # full error catch 
    # ------------------------------
    try:

        # ------------------------------
        # set up cameras 
        # ------------------------------

        # cameras variables
        left_camera_source = 2
        right_camera_source = 4
        pixel_width = 640
        pixel_height = 480
        angle_width = 78
        angle_height = 64 # 63
        frame_rate = 20
        camera_separation = 5 + 15/16

        # left camera 1
        ct1 = tt.Camera_Thread()
        ct1.camera_source = left_camera_source
        ct1.camera_width = pixel_width
        ct1.camera_height = pixel_height
        ct1.camera_frame_rate = frame_rate

        # right camera 2
        ct2 = tt.Camera_Thread()
        ct2.camera_source = right_camera_source
        ct2.camera_width = pixel_width
        ct2.camera_height = pixel_height
        ct2.camera_frame_rate = frame_rate

        # camera coding
        #ct1.camera_fourcc = cv2.VideoWriter_fourcc(*"YUYV")
        #ct2.camera_fourcc = cv2.VideoWriter_fourcc(*"YUYV")
        ct1.camera_fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        ct2.camera_fourcc = cv2.VideoWriter_fourcc(*"MJPG")

        # start cameras
        ct1.start()
        ct2.start()

        # ------------------------------
        # set up angles 
        # ------------------------------

        # cameras are the same, so only 1 needed
        angler = tt.Frame_Angles(pixel_width,pixel_height,angle_width,angle_height)
        angler.build_frame()

        # ------------------------------
        # set up motion detection 
        # ------------------------------

        # motion camera1
        # using default detect values
        targeter1 = tt.Frame_Motion()
        targeter1.contour_min_area = 1
        targeter1.targets_max = 1
        targeter1.target_on_contour = True # False = use box size
        targeter1.target_return_box = False # (x,y,bx,by,bw,bh)
        targeter1.target_return_size = True # (x,y,%frame)
        targeter1.contour_draw = True
        targeter1.contour_box_draw = False
        targeter1.targets_draw = True

        # motion camera2
        # using default detect values
        targeter2 = tt.Frame_Motion()
        targeter2.contour_min_area = 1
        targeter2.targets_max = 1
        targeter2.target_on_contour = True # False = use box size
        targeter2.target_return_box = False # (x,y,bx,by,bw,bh)
        targeter2.target_return_size = True # (x,y,%frame)
        targeter2.contour_draw = True
        targeter2.contour_box_draw = False
        targeter2.targets_draw = True

        # ------------------------------
        # stabilize 
        # ------------------------------

        # pause to stabilize
        time.sleep(0.5)

        # ------------------------------
        # targeting loop 
        # ------------------------------

        # variables
        maxsd = 2 # maximum size difference of targets, percent of frame
        klen  = 3 # length of target queues, positive target frames required to reset set X,Y,Z,D

        # target queues
        x1k,y1k,x2k,y2k = [],[],[],[]
        x1m,y1m,x2m,y2m = 0,0,0,0

        # last positive target
        # from camera baseline midpoint
        X,Y,Z,D = 0,0,0,0

        # loop
        while 1:

            # get frames
            frame1 = ct1.next(black=True,wait=1)
            frame2 = ct2.next(black=True,wait=1)

            # motion detection targets
            targets1 = targeter1.targets(frame1)
            targets2 = targeter2.targets(frame2)

            # check 1: motion in both frames
            if not (targets1 and targets2):
                x1k,y1k,x2k,y2k = [],[],[],[] # reset
            else:

                # split
                x1,y1,s1 = targets1[0]
                x2,y2,s2 = targets2[0]

                # check 2: similar size
                #if 100*(abs(s1-s2)/max(s1,s2)) > minsd:
                if abs(s1-s2) > maxsd:
                    x1k,y1k,x2k,y2k = [],[],[],[] # reset
                else:

                    # update queues
                    x1k.append(x1)
                    y1k.append(y1)
                    x2k.append(x2)
                    y2k.append(y2)

                    # check 3: queues full
                    if len(x1k) >= klen:

                        # trim
                        x1k = x1k[-klen:]
                        y1k = y1k[-klen:]
                        x2k = x2k[-klen:]
                        y2k = y2k[-klen:]

                        # mean values
                        x1m = sum(x1k)/klen
                        y1m = sum(y1k)/klen
                        x2m = sum(x2k)/klen
                        y2m = sum(y2k)/klen
                                
                        # get angles from camera centers
                        xlangle,ylangle = angler.angles_from_center(x1m,y1m,top_left=True,degrees=True)
                        xrangle,yrangle = angler.angles_from_center(x2m,y2m,top_left=True,degrees=True)
                        
                        # triangulate
                        X,Y,Z,D = angler.location(camera_separation,(xlangle,ylangle),(xrangle,yrangle),center=True,degrees=True)
        
            # display camera centers
            angler.frame_add_crosshairs(frame1)
            angler.frame_add_crosshairs(frame2)

            # display coordinate data
            fps1 = int(ct1.current_frame_rate)
            fps2 = int(ct2.current_frame_rate)
            text = 'X: {:3.1f}\nY: {:3.1f}\nZ: {:3.1f}\nD: {:3.1f}\nFPS: {}/{}'.format(X,Y,Z,D,fps1,fps2)
            lineloc = 0
            lineheight = 30
            for t in text.split('\n'):
                lineloc += lineheight
                cv2.putText(frame1,
                            t,
                            (10,lineloc), # location
                            cv2.FONT_HERSHEY_PLAIN, # font
                            #cv2.FONT_HERSHEY_SIMPLEX, # font
                            1.5, # size
                            (0,255,0), # color
                            1, # line width
                            cv2.LINE_AA, #
                            False) #

            # display current target
            if x1k:
                targeter1.frame_add_crosshairs(frame1,x1m,y1m,48)            
                targeter2.frame_add_crosshairs(frame2,x2m,y2m,48)            

            # display frame
            cv2.imshow("Left Camera 1",frame1)
            cv2.imshow("Right Camera 2",frame2)

            # detect keys
            key = cv2.waitKey(1) & 0xFF
            if cv2.getWindowProperty('Left Camera 1',cv2.WND_PROP_VISIBLE) < 1:
                break
            elif cv2.getWindowProperty('Right Camera 2',cv2.WND_PROP_VISIBLE) < 1:
                break
            elif key == ord('q'):
                break
            elif key != 255:
                print('KEY PRESS:',[chr(key)])

    # ------------------------------
    # full error catch 
    # ------------------------------
    except:
        print(traceback.format_exc())

    # ------------------------------
    # close all
    # ------------------------------

    # close camera1
    try:
        ct1.stop()
    except:
        pass

    # close camera2
    try:
        ct2.stop()
    except:
        pass

    # kill frames
    cv2.destroyAllWindows()

    # done
    print('DONE')

# ------------------------------
# run
# ------------------------------

if __name__ == '__main__':
    run()

# ------------------------------
# end
# ------------------------------
